In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install transformers
!pip install simpletransformers

In [ ]:
# install simpleT5
!pip install simplet5

     |████████████████████████████████| 808kB 3.8MB/s 
     |████████████████████████████████| 645kB 7.7MB/s 
     |████████████████████████████████| 10.6MB 9.9MB/s 
     |████████████████████████████████| 276kB 38.3MB/s 
     |████████████████████████████████| 829kB 42.2MB/s 
     |████████████████████████████████| 12.2MB 36.5MB/s 
     |████████████████████████████████| 4.5MB 41.4MB/s 
     |████████████████████████████████| 1.3MB 34.4MB/s 
     |████████████████████████████████| 296kB 40.3MB/s 
     |████████████████████████████████| 143kB 50.6MB/s 
  Created wheel for simplet5: filename=simplet5-0.0.7-cp37-none-any.whl size=5911 sha256=a94210e5e787effddad4d09ca2a18de9bf96e4b78b6b3bfea919c90a8d7a3de3
  Stored in directory: /root/.cache/pip/wheels/ff/8a/f5/359a3cfbdd545ccd115e7e520a0a8713d36060c8e82fe9325c
  Created wheel for fastt5: filename=fastt5-0.0.5-cp37-none-any.whl size=14574 sha256=a1b448c652b22e4e61f826330b405db6368e7a4f38b934411e270a94a17446cb
  Stored in directory: /root/

In [ ]:
import numpy as np
import pandas as pd
import os, json, gc, re, random
from tqdm.notebook import tqdm
# from tqdm import tqdm
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
import torch, transformers, tokenizers
torch.__version__, transformers.__version__, tokenizers.__version__

('1.8.1+cu101', '4.6.1', '0.10.3')

In [ ]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


##MOH-X - Abstract Summarization (Noun+Verb)

In [ ]:
data_file = pd.read_csv("data/trofix.csv")

In [ ]:
data_file.head()

,arg1,arg2,verb,sentence,verb_stem,label
0,mileage,struck,blow,Triple mileage has struck another blow to the ...,strike,1
1,terrorist,attack,target,U.S. officials said evidence suggests that a J...,attack,0
2,forces,stepped,use,"Some police forces , for example , have steppe...",step,0
3,day,pour,stream,"Every day his troops gather under the green , ...",pour,0
4,manufacturers,rolling,products,He says manufacturers are increasingly rolling...,roll,1


In [ ]:
data_file = data_file.rename(columns={'sentence': 'source_text'}) #abstract - sentence
data_file['target_text'] = data_file[['arg1', 'arg2', 'verb']].agg(' '.join, axis=1) #title - metaphorical words
#data_file['target_text'] = data_file['verb'] #title - metaphorical words

In [ ]:
papers = data_file[["source_text", "target_text"]]
papers.head()

,source_text,target_text
0,Triple mileage has struck another blow to the ...,mileage struck blow
1,U.S. officials said evidence suggests that a J...,terrorist attack target
2,"Some police forces , for example , have steppe...",forces stepped use
3,"Every day his troops gather under the green , ...",day pour stream
4,He says manufacturers are increasingly rolling...,manufacturers rolling products


In [ ]:
# split the data into training and test
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(papers, test_size=0.1)

In [ ]:
# import
from simplet5 import SimpleT5

# instatntiate
model = SimpleT5()

# load
model.from_pretrained("t5","t5-base")

# train
model.train(train_df=train_df, eval_df=test_df, source_max_token_len=512, target_max_token_len=128, max_epochs=5, batch_size=8, use_gpu=True)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42
INFO:filelock:Lock 140205605145232 acquired on /root/.cache/huggingface/transformers/684a47ca6257e4ca71f0037771464c5b323e945fbc58697d2fad8a7dd1a2f8ba.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d.lock


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

INFO:filelock:Lock 140205605145232 released on /root/.cache/huggingface/transformers/684a47ca6257e4ca71f0037771464c5b323e945fbc58697d2fad8a7dd1a2f8ba.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d.lock
INFO:filelock:Lock 140205584895952 acquired on /root/.cache/huggingface/transformers/90de37880b5ff5ac7ab70ff0bd369f207e9b74133fa153c163d14c5bb0116207.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529.lock


Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

INFO:filelock:Lock 140205584895952 released on /root/.cache/huggingface/transformers/90de37880b5ff5ac7ab70ff0bd369f207e9b74133fa153c163d14c5bb0116207.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529.lock
INFO:filelock:Lock 140205579741136 acquired on /root/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.66b9637a52aa11e9285cdd6e668cc0df14b3bcf0b6674cf3ba5353c542649637.lock


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
!ls outputs/

config.json			    SimpleT5-epoch-2-train-loss-0.2695
eval_results.txt		    SimpleT5-epoch-2-train-loss-0.2735
merges.txt			    SimpleT5-epoch-2-train-loss-0.3466
model_args.json			    SimpleT5-epoch-3-train-loss-0.0493
pytorch_model.bin		    SimpleT5-epoch-3-train-loss-0.0702
SimpleT5-epoch-0-train-loss-0.8219  SimpleT5-epoch-3-train-loss-0.1619
SimpleT5-epoch-0-train-loss-1.0937  SimpleT5-epoch-3-train-loss-0.1764
SimpleT5-epoch-0-train-loss-1.4472  SimpleT5-epoch-3-train-loss-0.1886
SimpleT5-epoch-0-train-loss-1.8205  SimpleT5-epoch-3-train-loss-0.2611
SimpleT5-epoch-0-train-loss-1.8353  SimpleT5-epoch-4-train-loss-0.037
SimpleT5-epoch-0-train-loss-1.8475  SimpleT5-epoch-4-train-loss-0.0388
SimpleT5-epoch-0-train-loss-1.915   SimpleT5-epoch-4-train-loss-0.1137
SimpleT5-epoch-1-train-loss-0.0803  SimpleT5-epoch-4-train-loss-0.1171
SimpleT5-epoch-1-train-loss-0.2455  SimpleT5-epoch-4-train-loss-0.1437
SimpleT5-epoch-1-train-loss-0.4197  SimpleT5-epoch-4-train-loss-0.2123
SimpleT5-epoch

In [ ]:
# load a trained model
model.load_model("outputs/SimpleT5-epoch-4-train-loss-1.1577", use_gpu=True)

In [ ]:
# let's see how it performerd:
sample_abstracts = test_df.sample(65)

for i, abstract in sample_abstracts.iterrows():
    print(f"===== Original Sentence =====")
    print(abstract['source_text'])
    summary= model.predict(abstract['source_text'])[0]
    print(f"\n===== Actual Metaphorical Tokens =====")
    print(f"{abstract['target_text']}")
    print(f"\n===== Predicted Metaphorical Tokens =====")
    print(f"{summary}")
    print("\n +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")

===== Original Sentence =====
His one spectacular crime came in the wee hours of Aug. 8 , 1963 , his 34th birthday , when he and his pals grabbed 120 sacks of well-worn bank notes being sent to London for pulping .

===== Actual Metaphorical Tokens =====
he grabbed sacks

===== Predicted Metaphorical Tokens =====
pals grabbed notes

 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

===== Original Sentence =====
The date would give two breweries , which now produce beer for export only , time to step up production , the bill said .

===== Actual Metaphorical Tokens =====
time step production

===== Predicted Metaphorical Tokens =====
time step production

 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

===== Original Sentence =====
Black & Decker Corp. stepped up pressure on American Standard Inc . , indicating that it might further sweeten its $ 65-a-share bid if financial information supports a higher figure .

====